In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import xmeans
import hcaa
import hrp
import hrb
from scipy.stats import skew, kurtosis

In [2]:
composition = pd.read_excel('data/raw/composicao_IBRx.xlsx')
# 2. Remove colunas "Company" e "Type"
composition = composition.drop(columns=["Company", "Type"])
# 3. Transforma de wide para long
date_cols = composition.columns.difference(['Code'], sort=False)
composition_long = composition.melt(
    id_vars='Code',
    value_vars=date_cols,
    var_name='Date',
    value_name='valores'
)
# 4. Converte as datas (formato "Dec-97") para datetime com mês e ano
composition_long['dates'] = pd.to_datetime(
    composition_long['Date'], format='%b-%y'
)
# 5. Remove a coluna original "Date" e reordena
composition_long = composition_long.drop(columns='Date')
composition_long = composition_long[['dates', 'Code', 'valores']]
# 6. Transforma de long para wide (tipo pivot_wider)
composition_wide = composition_long.pivot(
    index='dates',
    columns='Code',
    values='valores'
).reset_index()
composition = composition_wide.copy()

In [3]:
stocks  = pd.read_excel('data/raw/economatica_b3.xlsx')
# 2. Substituir os nomes dos meses em português por números
meses_pt = {
    "Jan": "01", "Fev": "02", "Mar": "03", "Abr": "04", "Mai": "05", "Jun": "06",
    "Jul": "07", "Ago": "08", "Set": "09", "Out": "10", "Nov": "11", "Dez": "12"
}
for pt, num in meses_pt.items():
    stocks["Data"] = stocks["Data"].str.replace(pt, num, regex=False)

# 3. Converter para datetime (assumindo formato "MM-YYYY" após substituições)
stocks["dates"] = pd.to_datetime(stocks["Data"], format="%m-%Y")

# 4. Reorganizar colunas e remover a original
stocks = stocks.drop(columns="Data")
stocks = stocks[["dates"] + [col for col in stocks.columns if col != "dates"]]

# 5. Filtrar intervalo de datas
stocks = stocks[(stocks["dates"] >= "1997-12-01") & (stocks["dates"] <= "2023-12-01")]

# 6. Renomear colunas removendo prefixo repetitivo
stocks.columns = stocks.columns.str.replace(
    r"Retorno\ndo fechamento\nem 1 meses\nEm moeda orig\najust p/ prov\n", "", regex=True
)
stocks.replace("-", np.nan, inplace=True)

In [4]:
InS = 120                           # tamanho janela 
OoS = stocks.shape[0] - InS         # janera de teste
p = stocks.shape[1] - 1             # numero de colunas
nmethods = 10                       # quantidade de simulações da matriz de cov (nao usado)

In [5]:
tickers = [i for i in range(p)]
w = []
Rport = pd.DataFrame(index=range(OoS), columns=['x_means', 'HCAA', 'HRP', 'HRB'])         # dataframe de retornos
to = pd.DataFrame(index=range(OoS - 1), columns=['x_means', 'HCAA', 'HRP', 'HRB'])        # dataframe de turnover
sspw = pd.DataFrame(index=range(OoS), columns=['x_means', 'HCAA', 'HRP', 'HRB'])          # dataframe de concentracao de pesos
w_xmeans_full = pd.DataFrame(index=range(OoS), columns=tickers)             # dataframe de pesos para seus tickers
w_hcaa_full = pd.DataFrame(index=range(OoS), columns=tickers)             # dataframe de pesos para seus tickers
w_hrp_full = pd.DataFrame(index=range(OoS), columns=tickers)         
w_hrb_full = pd.DataFrame(index=range(OoS), columns=tickers)         

In [6]:
def get_composition(composition: pd.DataFrame, i: int, InS: int, date_ins: pd.Series) -> pd.Series:
    # 1. Recorte das linhas
    df_slice = composition.iloc[i:(InS - 1 + i)]
    # 2. Filtra por data até a data final de interesse
    df_filtered = df_slice[df_slice["dates"] <= date_ins.iloc[-1]]
    # 3. Última linha do filtro
    last_row = df_filtered.tail(1)
    # 4. Remove a coluna "dates"
    last_row_no_dates = last_row.drop(columns="dates")
    # 5. Conta valores NaN por coluna
    return last_row_no_dates.isna().sum()

def calculate_to(previous_weights, desired_weights, oos_returns, p):
    # Substitui NaNs por 0 nos retornos
    oos_returns_ = oos_returns.fillna(0)

    # Atualiza os pesos com base nos retornos
    num = previous_weights * (1 + oos_returns_ / 100)
    den = np.nansum(num)  # soma ignorando NaNs

    updated_weights = num / den #if den != 0 else np.zeros_like(num)

    # Calcula o turnover como soma das diferenças absolutas
    t_o = np.sum(np.abs(desired_weights - updated_weights), axis=1)

    return t_o.tolist()[0]

for i in range(OoS):
    # obtendo as datas que serao usadas nessa janela
    date_ins = stocks.iloc[i:(InS - 1 + i), ]['dates']
    # indo na tabela composition que é a tabela das açoes mais liquidas do brasil
    # e pegando os ativos que estão dentro do intervalo de data e dentro da janela
    # a funcao get_composition retorna a soma de valores nan nessa janela
    aux1 = get_composition(composition, i, InS, date_ins)
    # indo na tabela de ações e pegando ativos dessa janela e removendo a coluna dates
    # para ter somente as colunas de retornos e calcula quantos valores nan tem essas
    # colunas de retorno 
    aux2 = stocks.iloc[i:(InS - 1 + i)].drop(columns="dates").isna().sum()
    # aqui pega as colunas onde não existem nan em aux1 e aux2, depois pega a intersecao
    aux = list(set(aux1[aux1 == 0].index).intersection(aux2[aux2 == 0].index))
    # pega os tickers que estao presente em aux que nao apresentam valores nan e então
    # vai na tabela de retornos e pega os retornos desses ativos, nessa janela
    retu_ins = stocks.iloc[i:(InS - 1 + i)][aux]#.to_numpy()
    # pega os retornos no mes seguinte dos ativos presentes em aux, esse retorno do mes
    # seguinte é o retorno fora da amostra que usaremos para calcular a performance do
    # portfolio
    r_oos = stocks.iloc[InS + i, ][aux].to_numpy()
    # guarda os retornos fora da amostra em um formato dataframe
    r_oos_full = pd.DataFrame(np.nan, index=[0], columns=stocks.columns[1:])
    r_oos_full.loc[0, aux] = r_oos
    # definindo uma seed diferente a cada iteração
    seed = np.random.seed(i)
    # lista contendo os ativos 
    asset = retu_ins.columns.tolist()
    print(f'começando backtest: {i}')
    # aqui performar o xmeans e pegar os pesos dos ativos
    w_xmeans = xmeans.main(retu_ins, np.cov(retu_ins, rowvar=False), asset, seed)
    w_hcaa   = hcaa.main(retu_ins, asset)
    w_hrp    = hrp.main(retu_ins)
    w_hrb    = hrb.main(retu_ins, asset)
    
    # lista para guardar em formato de dataframe os pesos que o xmeans retorna 
    w.append(pd.DataFrame([w_xmeans, w_hcaa, w_hrp]))
    
    # armazenando o retorno fora da amostra para o portfolio
    Rport.loc[i, :] = [(w_xmeans @ r_oos.T), (w_hcaa @ r_oos.T), (w_hrp @ r_oos.T), (w_hrb @ r_oos.T)]
    
    # armazenando os pesos elevado ao quadrado para medir a concentração dos pesos no portfólio
    sspw.loc[i, :]  = [np.sum(w_xmeans**2), np.sum(w_hcaa**2), np.sum(w_hrp**2), np.sum(w_hrb**2)]
    # armazenar no dataframe os pesos que o xmeans retornou, porém dessa vez 
    # vinculando o peso aos tickers
    w_xmeans_full.loc[i, aux] = w_xmeans
    w_hcaa_full.loc[i, aux] = w_hcaa
    w_hrp_full.loc[i, aux] = w_hrp
    w_hrb_full.loc[i, aux] = w_hrb

    if i > 2:
        # calculos de turnouver
        to_xmeans = calculate_to(w_xmeans_full.loc[i - 1, :],
            w_xmeans_full.loc[i, :],
            r_oos_full,
            p)
        to_hcaa = calculate_to(
            w_hcaa_full.loc[i - 1, :],
            w_hcaa_full.loc[i, :],
            r_oos_full,
            p
        )
        to_hrp = calculate_to(
            w_hrp_full.loc[i - 1, :],
            w_hrp_full.loc[i, :],
            r_oos_full,
            p
        )
        to_hrb = calculate_to(
            w_hrb_full.loc[i - 1, :],
            w_hrb_full.loc[i, :],
            r_oos_full,
            p
        )
        to.loc[i - 1, :] = [to_xmeans, to_hcaa, to_hrp, to_hrb]
        

começando backtest: 0
começando backtest: 1
começando backtest: 2
começando backtest: 3
começando backtest: 4
começando backtest: 5
começando backtest: 6
começando backtest: 7
começando backtest: 8
começando backtest: 9
começando backtest: 10
começando backtest: 11
começando backtest: 12
começando backtest: 13
começando backtest: 14
começando backtest: 15
começando backtest: 16
começando backtest: 17
começando backtest: 18
começando backtest: 19
começando backtest: 20
começando backtest: 21
começando backtest: 22
começando backtest: 23
começando backtest: 24
começando backtest: 25
começando backtest: 26
começando backtest: 27
começando backtest: 28
começando backtest: 29
começando backtest: 30
começando backtest: 31
começando backtest: 32
começando backtest: 33
começando backtest: 34
começando backtest: 35
começando backtest: 36
começando backtest: 37
começando backtest: 38
começando backtest: 39
começando backtest: 40
começando backtest: 41
começando backtest: 42
começando backtest: 4

In [14]:
def medidas(x, rf=0):
    x = pd.Series(x).astype(float)
    # Retorno médio e desvio padrão
    AV = x.mean()
    SD = x.std()
    # Sharpe Ratio
    SR = (AV - rf) / SD if SD != 0 else np.nan
    # Adjusted Sharpe Ratio
    sk = skew(x.dropna(), bias=False)
    kt = kurtosis(x.dropna(), fisher=True, bias=False)
    ASR = SR * (1 + (sk / 6) * SR - (kt / 24) * SR**2) if not np.isnan(SR) else np.nan
    # Sortino Ratio
    downside_diff = x - rf
    downside_risk = np.sqrt(np.mean(np.where(downside_diff < 0, downside_diff**2, 0)))
    SO = (AV - rf) / downside_risk if downside_risk != 0 else np.nan
    # Anualização
    output = [12 * AV, np.sqrt(12) * SD, np.sqrt(12) * SR, np.sqrt(12) * ASR, np.sqrt(12) * SO]
    return pd.Series(output, index=["AV", "SD", "SR", "ASR", "SO"])

linha_medidas = medidas(Rport['x_means'])
linha_to = pd.Series([to['x_means'].mean()], index=['TO'])
linha_sspw = pd.Series([sspw['x_means'].mean()], index=['SSPW'])

hcaa = medidas(Rport['HCAA'])
hcaa_to = pd.Series([to['HCAA'].mean()], index=['TO'])
hcaa_sspw = pd.Series([sspw['HCAA'].mean()], index=['SSPW'])

hrp = medidas(Rport['HRP'])
hrp_to = pd.Series([to['HRP'].mean()], index=['TO'])
hrp_sspw = pd.Series([sspw['HRP'].mean()], index=['SSPW'])

hrb = medidas(Rport['HRB'])
hrb_to = pd.Series([to['HRB'].mean()], index=['TO'])
hrb_sspw = pd.Series([sspw['HRB'].mean()], index=['SSPW'])

xmeans_results = pd.DataFrame(pd.concat([linha_medidas, linha_to, linha_sspw]), columns=["X_Means"]).T
hcaa_results = pd.DataFrame(pd.concat([hcaa, hcaa_to, hcaa_sspw]), columns=["HCAA"]).T
hrp_results = pd.DataFrame(pd.concat([hrp, hrp_to, hrp_sspw]), columns=["HRP"]).T
hrb_results = pd.DataFrame(pd.concat([hrb, hrb_to, hrb_sspw]), columns=["HRB"]).T
oos_results = pd.concat([xmeans_results, hcaa_results, hrp_results, hrb_results])
oos_results


,AV,SD,SR,ASR,SO,TO,SSPW
X_Means,11.790893,20.578426,0.572974,0.575262,1.028105,0.662684,0.038237
HCAA,11.349303,22.362377,0.507518,0.510565,0.908499,0.628444,0.038892
HRP,11.598831,21.964383,0.528075,0.530079,0.934846,0.514685,0.025408
HRB,10.998324,29.413502,0.373921,0.373777,0.618554,0.142153,0.505496


In [8]:
Rport

,x_means,HCAA,HRP,HRB
0,-1.923461,-2.65056,-1.988766,-3.540058
1,-8.308498,-9.42843,-8.408174,-15.481615
2,NaN,NaN,NaN,NaN
3,-3.363226,-0.184757,-2.671191,-5.134477
4,9.568721,9.830402,9.288711,17.857334
...,...,...,...,...
188,-6.308761,-6.525121,-6.603392,-10.453795
189,-1.300004,-2.372905,-2.562822,-4.241327
190,-4.382526,-5.447209,-4.697949,-2.415591
191,16.694605,15.743198,16.542152,16.402783


In [9]:
to

,x_means,HCAA,HRP,HRB
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,0.285925,0.616985,0.411679,0.009768
3,0.630734,0.46818,0.437945,0.020104
4,0.515867,0.278584,0.251882,0.003263
...,...,...,...,...
187,0.737688,0.131769,0.330799,0.000754
188,0.607023,0.447893,0.532914,0.000168
189,0.658909,0.17489,0.34173,0.001097
190,0.646333,0.246291,0.517189,0.004691


In [10]:
sspw

,x_means,HCAA,HRP,HRB
0,0.034928,0.043152,0.027651,0.500365
1,0.033664,0.04361,0.026956,0.500488
2,0.03033,0.044792,0.027422,0.500126
3,0.026597,0.048553,0.029731,0.500125
4,0.042072,0.039032,0.029414,0.500002
...,...,...,...,...
188,0.026326,0.029243,0.021285,0.500105
189,0.037956,0.029045,0.020373,0.500105
190,0.021996,0.028936,0.020673,0.500046
191,0.042569,0.028999,0.020894,0.500095
